In [ ]:
#pip install requests

In [ ]:
#pip install simplejson

In [ ]:
#pip install matplotlib

In [ ]:
#pip install yfinance

In [ ]:
#pip install sqlalchemy

In [ ]:
#pip install pymysql

In [ ]:
#pip install pandas_datareader

In [ ]:
#pip install ta

In [ ]:
import importlib
import subprocess
import sys

def install_dependencies(packages):
    """
    Installs only the missing packages or reinstalls them if they already exist.

    Args:
        packages (dict): A dictionary where the key is the package name 
                         and the value is the version (optional).
    """
    for package, version in packages.items():
        try:
            # Check if the package is already installed
            importlib.import_module(package)
            print(f"{package} is already installed. Uninstalling and reinstalling...")
            # Uninstall the package if it's already installed
            #subprocess.check_call([sys.executable, "-m", "pip", "uninstall", f"{package}", "-y"])
        except ImportError:
            print(f"{package} is not installed. Installing now...")

        # Install the package (after uninstalling if it existed)
        if version:
            subprocess.check_call([sys.executable, "-m", "pip", "install", f"{package}=={version}", "--upgrade", "--no-cache-dir"])
        else:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package, "--upgrade", "--no-cache-dir"])

# Define your dependencies (key: package name, value: version or None for latest)
dependencies = {
    "requests": None,
    #"numpy": "1.24.4",
    "yfinance": None,
    "simplejson": None,
    "matplotlib": None,
    "sqlalchemy": None,
    "pymysql": None,
    "pandas_datareader": None,
    "ta": None   
}

# Install the required dependencies
install_dependencies(dependencies)



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
import json
from datetime import datetime,timedelta

In [ ]:
import requests

In [ ]:
import os
path = os.path.join('/home','jovyan','files', 'datasets')

In [ ]:
from datetime import datetime

def convert_to_date(date_string, date_format="%Y-%m-%d"):
    """
    Converts a date string to a datetime object.
    
    Args:
        date_string (str): The date as a string.
        date_format (str): The format of the date string (default is '%Y-%m-%d').
    
    Returns:
        datetime: A datetime object representing the date, or None if conversion fails.
    """
    try:
        return datetime.strptime(date_string, date_format)
    except ValueError:
        print(f"Error: '{date_string}' does not match the format '{date_format}'")
        return None



In [ ]:
import yfinance as yf



def fetch_daily_data(symbol, start_date, end_date):
    # Download historical data from Yahoo Finance
    ticker = yf.Ticker(symbol)  # Symbol is like 'BTC-USD', 'ETH-USD', etc.
    #path =f'/../datasets'
   
    # Fetch data for the specified date range
    data = ticker.history(start=start_date, end=end_date, interval="1d")
  
    if data.empty:
        print(f"No data available for {symbol} between {start_date} and {end_date}")
    else:
        # File name based on symbol and start date
        filename = f'Yahoo_Finance_{symbol}_{start_date.strftime("%Y_%m_%d")}.csv'

           
         # Convert 'Date' column to datetime if not already
        #data['NewDate'] = convert_to_date (data['Date'])

        # Process data
        data['vol_fiat'] = data['Volume'] * data['Close']  # Calculate fiat volume
        data['Provider'] = "Yahoo Finance"
        data['Asset'] = symbol
        
        # Save to CSV
        file_path = os.path.join(path, filename)
        data.to_csv(file_path, index=True)  # 'index=True' to include the date column
        print(f"Data saved to {filename}")



        

In [ ]:
if __name__ == "__main__":
    pair = "BTC-USD"
    start = datetime.strptime("2015-01-01","%Y-%m-%d")
    today = pd.to_datetime('today').normalize()
    while(start <= today):
        end = start + timedelta(days=180)
        fetch_daily_data(symbol=pair,start_date=start,end_date=end)
        start = end + timedelta(days=1)

In [ ]:
import glob

combined_df = ""

csv_files = glob.glob(f"{path}/Yahoo_Finance_BTC-USD*.csv")

dataframes = []

for file in csv_files:
    df = pd.read_csv(file)
    dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=True)

combined_df.drop_duplicates()

In [ ]:
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://<user>::<pwd>@172.28.0.2:3306/bitcoin_db')

In [ ]:
combined_df.to_sql('crypto', con=engine, if_exists='replace', index=False)

In [ ]:
#https://alternative.me/crypto/fear-and-greed-index/
import requests
import pandas as pd


url = 'https://api.alternative.me/fng/?limit=0'

response = requests.get(url)
response.raise_for_status()  
data = response.json()['data']
df = pd.DataFrame(data)
# Convert 'timestamp' to datetime and set timezone to UTC
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s', utc=True)
df.to_sql('FearGreedIndex', con=engine, if_exists='replace', index=False)

In [ ]:
query = """
SELECT `Date`, `Open`, High, Low, `Close`, Volume, cast(fgi.value AS UNSIGNED) as FGI
FROM bitcoin_db.crypto btc
LEFT JOIN bitcoin_db.FearGreedIndex fgi
ON btc.`Date` = fgi.`timestamp`
"""

data = pd.read_sql(query,con=engine).sort_values(by='Date')
data.head()

In [ ]:
# Calculate the average of the FGI column, excluding NaN values
fg_average = data['FGI'].mean()

# Fill NaN values in the FGI column with the calculated average
data['FGI'].fillna(fg_average, inplace=True)

In [ ]:
import ta

In [ ]:
#https://www.investopedia.com/terms/m/macd.asp
#macd_object = ta.trend.MACD(data['close'])
macd_object = ta.trend.MACD(data['Close'], window_slow = 26, window_fast = 12, window_sign = 9, fillna= True)
data['MACD'] = macd_object.macd()
data['MACD_Signal'] = macd_object.macd_signal()
data['MACD_Diff'] = macd_object.macd_diff()
#data.tail()

In [ ]:
#https://www.investopedia.com/terms/m/mfi.asp
mfi_indicator = ta.volume.MFIIndicator(high=data['High'], low=data['Low'], close=data['Close'], volume=data['Volume'], window=14)
data['mfi'] = mfi_indicator.money_flow_index()

In [ ]:
# RSI
rsi = ta.momentum.rsi(data['Close'], window=14, fillna=True)
data['RSI'] = rsi

In [ ]:
# EMA
ema_short = ta.trend.EMAIndicator(data['Close'], window=12, fillna=True)
ema_long = ta.trend.EMAIndicator(data['Close'], window=26, fillna=True)
data['EMA_Short'] = ema_short.ema_indicator()
data['EMA_Long'] = ema_long.ema_indicator()

In [ ]:
# Bollinger Bands
bollinger = ta.volatility.BollingerBands(data['Close'], window=20, window_dev=2, fillna=True)
data['Bollinger_Upper'] = bollinger.bollinger_hband()
data['Bollinger_Lower'] = bollinger.bollinger_lband()
data['Bollinger_Middle'] = bollinger.bollinger_mavg()

In [ ]:
# Stochastic Oscillator
stoch = ta.momentum.StochasticOscillator(data['High'], data['Low'], data['Close'], window=14, smooth_window=3, fillna=True)
data['Stochastic'] = stoch.stoch()
data['Stochastic_Signal'] = stoch.stoch_signal()

In [ ]:
# ATR
atr = ta.volatility.AverageTrueRange(data['High'], data['Low'], data['Close'], window=14, fillna=True)
data['ATR'] = atr.average_true_range()

In [ ]:
# Parabolic SAR
sar = ta.trend.PSARIndicator(data['High'], data['Low'], data['Close'], fillna=True)
data['SAR'] = sar.psar()

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Prepare the data (assuming 'data' is already pre-processed with features)
data['Date'] = pd.to_datetime(data['Date'], utc=True)
data['Target'] = data['Close'].shift(-1)
data = data.dropna(subset=['Target'])

# Select features (excluding 'Date')
features = [
     'Bollinger_Upper', 'Bollinger_Lower', 'Bollinger_Middle','FGI', 'Close'
]

X = data[features]
y = data['Target']

In [ ]:
# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [ ]:
# Standardize the features
scaler = StandardScaler()

# Fit the scaler on the training data and transform both train and test data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the model
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test_scaled)

In [ ]:
# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

# Make predictions for the next closing price
# Use the last row from X_test (or any new data)
latest_data = X_test.iloc[-1:]  # Keep it as a DataFrame (not numpy array)

# Apply the same scaling to the latest data
latest_data_scaled = scaler.transform(latest_data)  # The feature names are kept

# Make prediction for the next closing price
next_close_pred = model.predict(latest_data_scaled)
print(f"Predicted next closing price: {next_close_pred[0]}")

In [ ]:
# Extract the coefficients and the intercept
coefficients = model.coef_
intercept = model.intercept_

# Create a dictionary mapping feature names to their corresponding coefficients
feature_names = X.columns
equation = f"Predicted next closing price = {intercept:.2f}"

# Add each feature and its coefficient to the equation
for feature, coef in zip(feature_names, coefficients):
    equation += f" + ({coef:.2f}) * {feature}"

# Display the equation
print("Final equation for predicting the next closing price:")
print(equation)


In [ ]:
import matplotlib.pyplot as plt

# Create a DataFrame to store the actual vs predicted values
results = pd.DataFrame({
    'Date': X_test.index,
    'Actual': y_test,
    'Predicted': y_pred
})

# Plotting the actual vs predicted closing prices
plt.figure(figsize=(14, 7))
plt.plot(results['Date'], results['Actual'], label='Actual Closing Price', color='blue', alpha=0.7)
plt.plot(results['Date'], results['Predicted'], label='Predicted Closing Price', color='red', linestyle='--', alpha=0.7)

# Add titles and labels
plt.title('Actual vs Predicted Closing Prices', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Price (USD)', fontsize=14)
plt.legend()
plt.grid(True)

# Display the plot
plt.xticks(rotation=45)
plt.tight_layout()  # Adjust layout to fit labels
plt.show()


In [ ]:
import statsmodels.api as sm

# Store original column names
original_columns = X_train.columns

# Standardize the features (X_train_scaled) for fitting the model
X_with_intercept = sm.add_constant(X_train_scaled)  # Add intercept term to the features
y_train_array = y_train.values  # Convert to numpy array for Statsmodels compatibility

# Fit the model using Statsmodels
model_sm = sm.OLS(y_train_array, X_with_intercept)  # Ordinary Least Squares Regression
results = model_sm.fit()  # Fit the model

# Display the summary, which includes the p-values for each feature
summary = results.summary()

# Map p-values to the original feature names
p_values = results.pvalues[1:]  # Exclude the intercept (constant)
features_with_p_values = dict(zip(original_columns, p_values))

# Print the p-values for each feature and check significance
print("Feature p-values (human-readable labels):")
for feature, p_value in features_with_p_values.items():
    significance = "Significant" if p_value < 0.05 else "Not Significant"
    print(f"{feature}: {p_value:.4f} ({significance})")

# Optionally, print the full summary
print("\nFull summary with p-values:")
print(summary)


